# 风力涡轮机故障诊断报告

本报告基于提供的模块化 Python 代码，展示从数据加载、预处理、特征工程、模型训练到结果汇总的完整流程。

## 1. 配置参数加载

 `config.py` 中负责加载各项系统参数，后续可灵活调整。

In [ ]:
from src.main.analysis.config import *
print(f"SCADA 数据路径: {SCADA_DATA_PATH}")
print(f"故障数据路径: {FAULT_DATA_PATH}")
print(f"时间窗口 (前): {WINDOW_BEFORE} 小时, (后): {WINDOW_AFTER} 小时")

## 2. 数据加载与概览

`data_preprocessing` 模块的 `load_data()` 函数负责加载 SCADA 数据和故障数据，并查看基本信息。

In [ ]:
from src.main.analysis.data_preprocessing import load_data

scada_data, fault_data = load_data()

## 3. 数据预处理

解析时间格式，清洗缺失值和重复值，将故障时间标签映射到 SCADA 记录上。

In [ ]:
from src.main.analysis.data_preprocessing import preprocess_data

labeled_data, fault_distribution = preprocess_data(scada_data, fault_data)
print("故障分布:")
print(fault_distribution)

## 4. 特征工程

使用滑动窗口生成统计特征，并添加季节性特征。

In [ ]:
from src.main.analysis.feature_engineering import extract_features

features = extract_features(labeled_data)
print(f"特征维度: {features.shape}")

## 5. 准备训练数据

将故障标签编码，并拆分为特征矩阵和标签向量。

In [ ]:
from src.main.analysis.feature_engineering import prepare_features

x, y, label_encoder = prepare_features(features)
print(f"X 维度: {x.shape}, y 样本数: {len(y)}")

## 6. 数据分割

按照设定的测试集比例将数据拆分为训练集和测试集。

In [ ]:
from src.main.analysis.model_training import split_data

x_train, x_test, y_train, y_test = split_data(x, y)
print(f"训练集样本: {x_train.shape[0]}, 测试集样本: {x_test.shape[0]}")

## 7. 特征选择

使用随机森林评估特征重要性，并筛选重要特征。

In [ ]:
from src.main.analysis.feature_engineering import select_features

x_train_sel, x_test_sel, scaler, selector, important_features = select_features(x_train, y_train, x_test)
print(f"选择后特征数: {x_train_sel.shape[1]}")

## 8. 模型训练与评估

训练多种机器学习模型并比较其性能，取最佳。

In [ ]:
from src.main.analysis.model_training import train_models

best_model, best_model_name, best_acc, accuracies, y_test, y_pred = train_models(
    x_train_sel, y_train, x_test_sel, y_test
)
print(f"最佳模型: {best_model_name}, 准确率: {best_acc:.4f}")

## 9. 数据图像绘制

将原始数据，模型训练数据，模型训练成果整理并绘图

In [ ]:
from src.main.analysis.plot import graphics_drawing

graphics_drawing(labeled_data, fault_distribution, best_model, best_model_name, accuracies, y_test, y_pred)

## 10. 仪表盘

综合数据与图像呈现

In [ ]:
from src.main.analysis.dashboard import app_start
app_start()

## 11. 模型保存与结果汇总

保存最佳模型及预处理器，打印总结报告。

In [ ]:
from src.main.utils import save_model, print_summary

save_model(best_model, scaler, selector, label_encoder)
print_summary(
    fault_distribution, features.shape, x_train_sel.shape[1],
    important_features, best_model_name, best_acc, accuracies
)